In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# 모델 선택 및 평가 관련 라이브러리
from sklearn.model_selection import train_test_split #,GridSearchCV(하이퍼파라미터), cross_val_score(교차 검증)
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
                            #(accuracy는 맞춘 비율을 계산하여 정확도를 높이지만, 모델이 클래스 구분을 잘 했는지는 파악 불가
                            #roc는 확률값을 활용하여 모델의 성능 평가,, 클래스를 잘 구별하는 지 확인, 1에 가까울수록 좋은 모델
                                #accuracy는 데이터가 균형 잡혀 있을 때 G, roc는 데이터가 불균형할 때 G )

# 전처리: 범주형 인코딩, 결측치 대체, 스케일링
from sklearn.preprocessing import OrdinalEncoder       #범주형 데이터 숫자로 변환
from sklearn.preprocessing import StandardScaler       #데이터 표준화하여 평균 0, 표준편차 1로 변환, 데이터 크기 관련X 모델이 동일한 가중치 부여, (SVM, 로지스틱회귀, KNN, PCA 필수)
from sklearn.impute import SimpleImputer               #결측치 자동 대체 (strategy='mean':평균값, 'median':중앙값,'most_frequent':최빈값)


In [3]:
train = pd.read_csv('/content/drive/MyDrive/open/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/open/test.csv').drop(columns=['ID'])

In [4]:
# 타겟(입신 성공 여부) 분리 + 결측치 제거
y = train['임신 성공 여부']  # 타겟 변수 분리
x = train.drop(columns=['임신 성공 여부'])  # 입력 변수(x) 정의

# x에서 y의 결측치가 있는 행 제거
x = x.loc[y.dropna().index]
y = y.dropna()  # y에서도 결측치 제거


In [5]:
#범주형 컬럼 전체 정의
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

#범주형 컬럼을 문자열(str)로 변환
for col in categorical_columns:
  for col in categorical_columns:
    x.loc[:, col] = x[col].astype(str)
    test.loc[:, col] = test[col].astype(str)     #.loc[] 활용해야 안전

<ipython-input-5-e6a91ad6f264>:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '0' '1' ... '1' '1' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  x.loc[:, col] = x[col].astype(str)
<ipython-input-5-e6a91ad6f264>:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '0' '0' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test.loc[:, col] = test[col].astype(str)     #.loc[] 활용해야 안전
<ipython-input-5-e6a91ad6f264>:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  x.loc[:, col] = x[col].astype(str)
<ipython-input-5-e6a91ad6f264>:56: F

In [6]:
#수치형 컬럼 전체 정의
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [7]:
#train 데이터를 train/val로 분할
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [8]:
# for col in categorical_columns:
#     most_frequent_value = x_train[col].mode()[0]  # 최빈값
#     x_train[col].fillna(most_frequent_value, inplace=True)
#     x_val[col].fillna(most_frequent_value, inplace=True)
#     test[col].fillna(most_frequent_value, inplace=True)

In [9]:
# for col in categorical_columns:
#     freq = x_train[col].value_counts(normalize=True)  # 비율 계산
#     threshold = 0.01  # 1% 미만의 희귀값을 이상치로 간주
#     rare_values = freq[freq < threshold].index
#     x_train[col] = x_train[col].replace(rare_values, "Other")
#     x_val[col] = x_val[col].replace(rare_values, "Other")
#     test[col] = test[col].replace(rare_values, "Other")

In [10]:
#범주형 변수 인코딩
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
x_train_enc = x_train.copy()
x_train_enc[categorical_columns] = ordinal_encoder.fit_transform(x_train_enc[categorical_columns])

x_val_enc   = x_val.copy()
x_val_enc[categorical_columns]   = ordinal_encoder.transform(x_val_enc[categorical_columns])


In [11]:
# imputer = SimpleImputer(strategy='most_frequent')
# x_train_enc[numeric_columns] = imputer.fit_transform(x_train_enc[numeric_columns])
# x_val_enc[numeric_columns] = imputer.transform(x_val_enc[numeric_columns])

In [12]:
#수치형 변수 결측치 대체 (중앙값 사용)
imputer = SimpleImputer(strategy='median')
x_train_enc[numeric_columns] = imputer.fit_transform(x_train_enc[numeric_columns])
x_val_enc[numeric_columns]   = imputer.transform(x_val_enc[numeric_columns])


In [13]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.5 MB/s eta 0:00:00


In [73]:
from catboost import CatBoostClassifier

# CatBoost 모델 설정
cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=10,
    random_seed=100,
    loss_function='Logloss',
    verbose=10,
    #eval_metric= Binary f1_score,
    scale_pos_weight=3,
    one_hot_max_size=1
)


In [74]:

# 모델 학습 (early_stopping 적용)
cat_model.fit(
    x_train_enc, y_train,
    eval_set=[(x_val_enc, y_val)],
    early_stopping_rounds=10
)


0:	learn: 0.6892086	test: 0.6892787	best: 0.6892787 (0)	total: 142ms	remaining: 2m 22s
10:	learn: 0.6571971	test: 0.6578137	best: 0.6578137 (10)	total: 1.73s	remaining: 2m 35s
20:	learn: 0.6354031	test: 0.6364902	best: 0.6364902 (20)	total: 3.88s	remaining: 3m 1s
30:	learn: 0.6210995	test: 0.6225719	best: 0.6225719 (30)	total: 5.35s	remaining: 2m 47s
40:	learn: 0.6117737	test: 0.6136233	best: 0.6136233 (40)	total: 6.83s	remaining: 2m 39s
50:	learn: 0.6046114	test: 0.6068067	best: 0.6068067 (50)	total: 8.29s	remaining: 2m 34s
60:	learn: 0.5990416	test: 0.6015512	best: 0.6015512 (60)	total: 9.75s	remaining: 2m 30s
70:	learn: 0.5949322	test: 0.5977498	best: 0.5977498 (70)	total: 11.2s	remaining: 2m 26s
80:	learn: 0.5911194	test: 0.5941979	best: 0.5941979 (80)	total: 12.6s	remaining: 2m 23s
90:	learn: 0.5882795	test: 0.5915728	best: 0.5915728 (90)	total: 14.5s	remaining: 2m 24s
100:	learn: 0.5859213	test: 0.5894675	best: 0.5894675 (100)	total: 16.3s	remaining: 2m 25s
110:	learn: 0.5840609	

In [75]:
# 검증 데이터 ROC-AUC 평가
val_pred_proba = cat_model.predict_proba(x_val_enc)[:, 1]
val_roc_auc = roc_auc_score(y_val, val_pred_proba)
print("cat Validation ROC-AUC:", val_roc_auc)

cat Validation ROC-AUC: 0.7374317559104839


In [76]:
#전체 학습 데이터 전처리 적용
x_full_enc = x.copy()
x_full_enc[categorical_columns] = ordinal_encoder.transform(x_full_enc[categorical_columns])  # fit X
x_full_enc[numeric_columns] = imputer.transform(x_full_enc[numeric_columns])  # fit X

In [77]:
#최종 XGBoost 모델 (단, early stopping 없이 기본 n_estimators 사용)
model_full = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=10,
    random_seed=100,
    loss_function='Logloss',
    verbose=10,
    #eval_metric= Binary f1_score,
    scale_pos_weight=3,
    one_hot_max_size=1
)

model_full.fit(x_full_enc, y)

0:	learn: 0.6892456	total: 212ms	remaining: 3m 31s
10:	learn: 0.6581146	total: 1.94s	remaining: 2m 54s
20:	learn: 0.6364030	total: 3.69s	remaining: 2m 52s
30:	learn: 0.6216175	total: 6.19s	remaining: 3m 13s
40:	learn: 0.6111211	total: 8.12s	remaining: 3m 9s
50:	learn: 0.6041628	total: 10.7s	remaining: 3m 19s
60:	learn: 0.5988305	total: 12.4s	remaining: 3m 11s
70:	learn: 0.5947744	total: 14.2s	remaining: 3m 6s
80:	learn: 0.5915959	total: 16s	remaining: 3m 1s
90:	learn: 0.5889339	total: 18.5s	remaining: 3m 5s
100:	learn: 0.5865037	total: 20.4s	remaining: 3m 1s
110:	learn: 0.5846020	total: 22.1s	remaining: 2m 57s
120:	learn: 0.5830530	total: 23.9s	remaining: 2m 53s
130:	learn: 0.5817585	total: 25.7s	remaining: 2m 50s
140:	learn: 0.5804962	total: 27.4s	remaining: 2m 47s
150:	learn: 0.5794852	total: 30s	remaining: 2m 48s
160:	learn: 0.5786231	total: 32.5s	remaining: 2m 49s
170:	learn: 0.5778266	total: 36.2s	remaining: 2m 55s
180:	learn: 0.5770870	total: 38.6s	remaining: 2m 54s
190:	learn: 0

In [78]:
#전체 학습 데이터 전처리 적용
x_full_enc = x.copy()
x_full_enc[categorical_columns] = ordinal_encoder.transform(x_full_enc[categorical_columns])  # fit X
x_full_enc[numeric_columns] = imputer.transform(x_full_enc[numeric_columns])  # fit X

In [79]:
#테스트 데이터 전처리 및 예측
x_test_enc = test.copy()
x_test_enc[categorical_columns] = ordinal_encoder.transform(x_test_enc[categorical_columns])
x_test_enc[numeric_columns] = imputer.transform(x_test_enc[numeric_columns])
test_pred_proba = model_full.predict_proba(x_test_enc)[:, 1]

In [80]:
# 제출 파일 생성
sample_submission = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')

sample_submission['probability'] = test_pred_proba
sample_submission.to_csv('./0226_cat.csv', index=False)
print("Finish! Submission file saved as 0226_cat.csv")

Finish! Submission file saved as 0226_cat.csv
